In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import os
import zipfile
from google.colab import files
!pip install split-folders
import splitfolders
from tensorflow.keras.optimizers import RMSprop

In [ ]:
!unzip drive/"My Drive"/ArchiveNI.zip

Archive:  drive/My Drive/ArchiveNI.zip
replace data/natural_images/airplane/airplane_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
splitfolders.ratio('/content/natural_images', output="splitnaturalimages", seed=1337, ratio=(0.7, 0.2,0.1))

Copying files: 6899 files [00:04, 1481.13 files/s]


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
        '/content/splitnaturalimages/train', # This is the source directory for training images
        target_size=(224, 224), #All images will be resized to 224*224
        batch_size=64,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1/255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                 horizontal_flip=True,
                                 vertical_flip=True)


validation_generator = validation_datagen.flow_from_directory(
        '/content/splitnaturalimages/val',  # This is the source directory for validation images
        target_size=(224, 224),  # All images will be resized to 224*224
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1/255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)


test_generator = test_datagen.flow_from_directory(
        '/content/splitnaturalimages/test',  # This is the source directory for test images
        target_size=(224, 224),  # All images will be resized to 224*224
        batch_size=32,
        class_mode='categorical')

Found 4826 images belonging to 8 classes.
Found 1377 images belonging to 8 classes.
Found 696 images belonging to 8 classes.


In [ ]:
class_names = train_generator.class_indices
print(class_names)

{'airplane': 0, 'car': 1, 'cat': 2, 'dog': 3, 'flower': 4, 'fruit': 5, 'motorbike': 6, 'person': 7}


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=40,
      steps_per_epoch=47,
      validation_steps=40,
      verbose=2)

Epoch 1/40
47/47 - 123s - loss: 2.0597 - accuracy: 0.3051 - val_loss: 1.3876 - val_accuracy: 0.5398 - 123s/epoch - 3s/step
Epoch 2/40
47/47 - 122s - loss: 1.3954 - accuracy: 0.4894 - val_loss: 1.1645 - val_accuracy: 0.5727 - 122s/epoch - 3s/step
Epoch 3/40
47/47 - 121s - loss: 1.1851 - accuracy: 0.5902 - val_loss: 0.9052 - val_accuracy: 0.6586 - 121s/epoch - 3s/step
Epoch 4/40
47/47 - 120s - loss: 1.0153 - accuracy: 0.6401 - val_loss: 0.9769 - val_accuracy: 0.6492 - 120s/epoch - 3s/step
Epoch 5/40
47/47 - 121s - loss: 0.9418 - accuracy: 0.6609 - val_loss: 0.8419 - val_accuracy: 0.7023 - 121s/epoch - 3s/step
Epoch 6/40
47/47 - 120s - loss: 0.8577 - accuracy: 0.6936 - val_loss: 0.7297 - val_accuracy: 0.7484 - 120s/epoch - 3s/step
Epoch 7/40
47/47 - 120s - loss: 0.8059 - accuracy: 0.7091 - val_loss: 0.7458 - val_accuracy: 0.7242 - 120s/epoch - 3s/step
Epoch 8/40
47/47 - 122s - loss: 0.7665 - accuracy: 0.7202 - val_loss: 0.8400 - val_accuracy: 0.6992 - 122s/epoch - 3s/step
Epoch 9/40
47/47

In [ ]:
model.evaluate(test_generator)

22/22 [==============================] - 12s 541ms/step - loss: 0.4944 - accuracy: 0.8520


[0.494366317987442, 0.852011501789093]